In [54]:
from dotenv import load_dotenv
load_dotenv()

True

In [55]:
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.schema import HumanMessage, AIMessage
from langchain.memory import ConversationBufferMemory
from langgraph.graph import StateGraph, END
from typing import TypedDict, List, Annotated
from langchain.schema import BaseMessage

import os

print(os.environ["OPENAI_API_KEY"])


sk-proj-4J7r72c5kPr0nk6Cjyy3T3BlbkFJTT8sBZM0gk7XWxwCmzgU


In [56]:
EXPERT_TYPES = [
    "Mathematician",
    "Physicist",
    "Computer Scientist",
    "Historian",
    "Linguist",
    "Philosopher",
    "Biologist",
    "Chemist",
    "Economist",
    "Psychologist"
]

# Create the language model
llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)

meta_prompt = """You are Meta-Expert, an extremely clever expert with the unique ability to collaborate with multiple experts to tackle any task and solve complex problems. Your role is to oversee the communication between the experts, effectively using their skills to answer a given question while applying your own critical thinking and verification abilities.

To communicate with an expert, type the expert type (e.g., "Mathematician", "Physicist", etc.), followed by a colon ":", and then provide a detailed instruction enclosed within triple quotes.

Available expert types: {expert_types}

Current task: {task}
Current conversation history: {history}
Current turn: {turn_count}

What is your next action? If you have sufficient information to answer the question, or if you've reached turn 5, you must provide a final answer. Present it as follows:
>> FINAL ANSWER: '''[final answer]'''
>> CONFIDENCE: [score between 0 and 1]

If you need more information, consult an expert. But remember, you must provide a final answer by turn 5 at the latest.
"""

meta_prompt_template = ChatPromptTemplate.from_template(meta_prompt)



expert_prompt = """You are an expert {expertise}. Your role is to provide accurate and insightful information in your field of expertise. Please respond to the following instruction:

{instruction}

Provide your response based on your expertise as a {expertise}. Be thorough and precise in your explanation."""

expert_prompt_template = ChatPromptTemplate.from_template(expert_prompt)



In [57]:

from typing import TypedDict, List, Literal

class State(TypedDict):
    task: str
    history: List[BaseMessage]
    response: str
    next: Literal["call_meta_model", "call_expert", "end"]
    turn_count: int

def call_meta_model(state: State) -> State:
    history = state["history"]
    task = state["task"]
    turn_count = state["turn_count"] + 1
    
    messages = meta_prompt_template.format_messages(
        task=task, 
        history=history, 
        expert_types=", ".join(EXPERT_TYPES),
        turn_count=turn_count
    )
    response = llm(messages)
    
    new_state = {
        "response": response.content, 
        "history": history + [AIMessage(content=response.content)], 
        "task": task,
        "next": "call_expert",
        "turn_count": turn_count
    }
    
    if ">> FINAL ANSWER:" in response.content or turn_count >= 5:
        new_state["next"] = "end"
    
    return new_state

def call_expert(state: State) -> State:
    response = state["response"]
    history = state["history"]
    task = state["task"]
    turn_count = state["turn_count"]
    
    for expert_type in EXPERT_TYPES:
        if f"{expert_type}:" in response:
            expert, instruction = response.split(":", 1)
            instruction = instruction.strip().strip('"""')
            
            messages = expert_prompt_template.format_messages(expertise=expert_type, instruction=instruction)
            expert_response = llm(messages)
            
            return {
                "response": expert_response.content, 
                "history": history + [AIMessage(content=expert_response.content)], 
                "task": task,
                "next": "call_meta_model",
                "turn_count": turn_count
            }
    
    # If no expert type is found, return to meta model
    return {**state, "next": "call_meta_model"}



In [58]:

workflow = StateGraph(State)

# Add nodes
workflow.add_node("call_meta_model", call_meta_model)
workflow.add_node("call_expert", call_expert)

# Set entry point
workflow.set_entry_point("call_meta_model")

# Add edges
workflow.add_edge("call_meta_model", "call_expert")
workflow.add_edge("call_expert", "call_meta_model")

# Add conditional edges based on the 'next' key
workflow.add_conditional_edges(
    "call_meta_model",
    lambda x: x["next"],
    {
        "call_expert": "call_expert",
        "end": END
    }
)

workflow.add_conditional_edges(
    "call_expert",
    lambda x: x["next"],
    {
        "call_meta_model": "call_meta_model",
    }
)

app = workflow.compile()

In [60]:
task = "Explain the philosophical implications of quantum entanglement and its potential impact on our understanding of free will."
initial_state = State(task=task, history=[], response="", next="call_meta_model", turn_count=0)

try:
    for output in app.stream(initial_state):
        node_name = list(output.keys())[0]
        node_output = output[node_name]
        
        print(f"\n{node_name} : {node_output['response']}")
        
        if node_output['next'] == "end":
            final_answer = node_output['response'].split(">> FINAL ANSWER:")[-1].split(">> CONFIDENCE:")[0].strip().strip("'''")
            # confidence = float(node_output['response'].split(">> CONFIDENCE:")[-1].strip())
            confidence = node_output['response'].split(">> CONFIDENCE:")[-1].strip()
            print(f"\nFinal Answer: {final_answer}")
            print(f"Confidence: {confidence}")

    print("\nMeta-prompting process completed.")
except Exception as e:
    print(f"An error occurred: {str(e)}")
    print(f"Output that caused the error: {output}")


call_meta_model : I will start by consulting the Philosopher expert to gain insights into the philosophical implications of quantum entanglement and its potential impact on our understanding of free will.

Philosopher: "Can you explain the philosophical implications of quantum entanglement and its potential impact on our understanding of free will?"


call_expert : Quantum entanglement is a phenomenon in quantum physics where two or more particles become connected in such a way that the state of one particle is dependent on the state of the other, regardless of the distance between them. This means that the properties of one particle are instantly correlated with the properties of the other, even if they are separated by vast distances. This concept challenges our traditional understanding of causality and locality in physics.

From a philosophical perspective, quantum entanglement raises questions about the nature of reality and the relationship between physical systems. It suggests 